In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("Dataset_1_selected_columns.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815935 entries, 0 to 815934
Data columns (total 18 columns):
Unnamed: 0                  815935 non-null int64
id                          815935 non-null int64
doctorId                    815935 non-null int64
address_city                815787 non-null object
colleagueRecommendations    815623 non-null float64
gender                      657752 non-null object
grade                       815623 non-null float64
practitionerTypes           701759 non-null object
recommendations             325759 non-null object
views                       815935 non-null int64
furtherEducation            235378 non-null object
hasMultipleLocations        815935 non-null int64
name_title                  367813 non-null object
otvCustomerType             657752 non-null object
positiveSnippets            148886 non-null object
reviewsCount                781107 non-null float64
likes                       451998 non-null float64
services               

# Using regression model to understand the relation between Grade(dependent) and other variables (independent)

In [ ]:
graded_docs = data[data["grade"].notnull()]
# len(graded_docs) 815623

X = graded_docs[["colleagueRecommendations", "gender","recommendations", "views", #"furtherEducation", 
          "hasMultipleLocations", "name_title", "otvCustomerType", "reviewsCount","likes"]]
y = graded_docs["grade"]

In [ ]:
# print("Missing values count: ",X["colleagueRecommendations"].isna().sum())
X["colleagueRecommendations"] = X["colleagueRecommendations"].fillna(0)


X.gender.unique()
X['gender'] = X['gender'].fillna(0)
# X.tail()
# X['gender'].isnull().values.any()


def gender_to_numeric(a):
    if a=='FEMALE':
        return 2
    elif a=='MALE':
        return 1
    else:
        return 0
  

X['gender'] = X['gender'].apply(gender_to_numeric)
# X.head()
# X.gender.unique()


X['recommendations'] = X['recommendations'].str.rstrip('%').astype('float')*100 /100.0 
X['recommendations'] = X['recommendations'].fillna(0.0)
# X.tail()
# X['recommendations'].isnull().values.any()


X['views'].isnull().values.any()


# X['furtherEducation'] = X['furtherEducation'].fillna(0)
# X['furtherEducation'].isnull().values.any()
# X.furtherEducation.unique() ### was i supposed to change this to numeric???

X['hasMultipleLocations'].isnull().values.any()


X.name_title.unique()

X['name_title'] = X['name_title'].fillna('NoData')
X['name_title'].isnull().values.any()


def title_to_numeric(b):
    if 'Dr.' in b:
        return 2
    elif 'NoData' in b:
        return 0
    else:
        return 1

    ### giving all of the people having Dr. 2 and those who dont have 1 and any null vaulues have been converted to NoData and 
    ### given value of 0
    
X['name_title'] = X['name_title'].apply(title_to_numeric)

print("patient type :\n", X.otvCustomerType.unique())

X['otvCustomerType'] = X['otvCustomerType'].fillna(0)
X['otvCustomerType'].isnull().values.any()

def otv_to_numeric(c):
    if c=='kein-OTV-Kunde':
        return 1
    elif c=='OTV-inaktiv':
        return 2
    elif c=='OTV-aktiv':
        return 3
    elif c=='OTV-inaktiv|Kunde-neu':
        return 4
    elif c=='OTV-aktiv|Kunde-neu':
        return 5
    else:
        return 0

X['otvCustomerType'] = X['otvCustomerType'].apply(otv_to_numeric)
# X.otvCustomerType.unique()

X['reviewsCount'] = X['reviewsCount'].fillna(0)

print("Any null values? : ", X['likes'].isnull().values.any())

X['likes'] = X['likes'].fillna(0)
# X.head()

X.isna().any()
y.isna().any()

patient type :
 ['kein-OTV-Kunde' 'OTV-inaktiv' 'OTV-aktiv' 'OTV-inaktiv|Kunde-neu'
 'OTV-aktiv|Kunde-neu' nan]
Any null values? :  True


False

In [ ]:
# reg = LinearRegression(fit_intercept = False).fit(X, y)
model = LinearRegression().fit(X, y)
model.coef_

from regressors import stats
stats.summary(model, X, y, X.columns)

Residuals:
    Min      1Q  Median      3Q      Max
-5.9226 -0.0923  0.3273  0.4258  30.3143


Coefficients:
                          Estimate  Std. Error   t value   p value
_intercept                0.408195    0.002145  190.2941  0.000000
colleagueRecommendations -0.133389    0.002880  -46.3085  0.000000
gender                   -0.003998    0.001604   -2.4925  0.012686
recommendations           0.009608    0.000027  359.6027  0.000000
views                     0.000007    0.000000   78.2771  0.000000
hasMultipleLocations     -0.130537    0.003565  -36.6184  0.000000
name_title                0.105949    0.001167   90.7970  0.000000
otvCustomerType          -0.083549    0.003169  -26.3609  0.000000
reviewsCount              0.007097    0.000066  107.2577  0.000000
likes                    -0.000206    0.000022   -9.3524  0.000000
---
R-squared:  0.21291,    Adjusted R-squared:  0.21290
F-statistic: 24514.20 on 9 features


#### Positive Coefficients:
     1. "recommendations"
     2. "reviewsCount"
     3. "views"
     4. "gender"
    
#### Negative Coefficients:
    ## "colleagueRecommendations"
    ## "hasMultipleLocations"
    ## "name_title"
    ## "otvCustomerType"
    ## "likes"

# Conclusion: 

<b>Recommendations</b> by other patients, <b>Reviews</b> and <b>Views</b> on the doctor profile - have a major impact on <b>patient's satisfaction! </b> <i><b>(measured in terms of grades given by them to doctor)</b></i>